In [1]:
# suppress warnings
import warnings

warnings.filterwarnings('ignore')


In [2]:
import nannyml as nml


In [3]:
print(nml.__path__)


['/Users/carterblair/NannyMLdev/nannyml/nannyml']


In [4]:
ref_df, ana_df, _ = nml.datasets.load_synthetic_binary_classification_dataset()


In [6]:
cbpe = nml.CBPE(
    y_pred_proba='y_pred_proba',
    y_pred='y_pred',
    y_true='work_home_actual',
    problem_type='classification_binary',
    metrics=['roc_auc', 'f1', 'precision', 'recall', 'specificity', 'accuracy', 'confusion_matrix'],
    normalize_confusion_matrix=None,
    chunk_size=20_000,
).fit(ref_df)
result = cbpe.estimate(ana_df)
df = result.to_df()

columns = []
for m in result.metrics:
    if hasattr(m, 'components'):
        for c in m.components:
            columns.append((c))
    else:
        columns.append((m.column_name))

sut = result.filter(period='analysis').to_df()[[('chunk', 'key')] + [(name, 'value') for name in columns]]


print("(")
print("\t\t\t{'chunk_size\':20000, \'cm_normalization\' = None},")
print("\t\t\tpd.DataFrame(")
print("\t\t\t\t{")
# print the chunk keys in this format: 'key': ['[0:19999]', '[20000:49999]']
print("\t\t\t\t\t\'key\'", ":", sut['chunk']['key'].unique().tolist(), ",")

# print the column names and the list of values in this format: 'roc_auc': [0.5, 0.5] using f strings
for col in sut.columns:
    if col[0] != 'chunk':
        print(f'\t\t\t\t\t\'estimated_{col[0]}\': {sut[col].values.tolist()},')

print("\t\t\t\t}")
print("\t\t\t),")
print("\t\t),")


(
			{'chunk_size':20000, 'cm_normalization' = None},
			pd.DataFrame(
				{
					'key' : ['[0:19999]', '[20000:49999]'] ,
					'estimated_roc_auc': [0.9690827692140925, 0.9627722759203226],
					'estimated_f1': [0.9479079222515973, 0.9278089207836576],
					'estimated_precision': [0.9436121782324026, 0.9197836255005452],
					'estimated_recall': [0.9522429574319092, 0.9359754933378336],
					'estimated_specificity': [0.9434949869571513, 0.9123732942949082],
					'estimated_accuracy': [0.9478536003143163, 0.9245926106862006],
					'estimated_true_positive': [9488.96406430504, 14537.180201036117],
					'estimated_true_negative': [9468.107941981285, 13200.5981195499],
					'estimated_false_positive': [567.0359356949585, 1267.8197989638852],
					'estimated_false_negative': [475.8920580187156, 994.4018804500995],
				}
			),
		),
